In [1]:
def AskUserWhatToDo():
    choice=0
    loss_history=[]
    accuracy_history=[]
    while (choice == 0):
        print("\n\nSelect an option by typing the Indicated letter and pressing  \"Enter\" \n\n")
        print("a:  Load csv of data to model")
        print("b: Partition  loaded data into Training, Testing, and Validation")
        print("c: Manually Define MLP Architecture which is compatible with loaded data")
        print("d: Load MLP Architecture From Previous Session which is compatible with loaded data")
        print("e: Initialize weights and biases for use with training")
        print("f: Load weights and biases from previous session")
        print("g: Save Current Architecture For Future Session")
        print("h: Save Current Weights For Future Session")
        print("i: Choose Training Parameters")
        print("j: Save Training Parameters")
        print("k: Load Training Parameters")
        print("l: Quit Program Without Saving")
        print("m: Train Model")
        print("n: Test Models")
        print("o: Print Accuracy History")
        print("p: Print Loss History")
    
        x=input("Your Choice: ")
    
        if (x=='a'):
            current_loaded_data=load_data()
            current_data_size=current_loaded_data.shape[1]-1
        elif (x=='b'):
            current_loaded_data=partition_current_loaded_data(current_loaded_data)
            current_data_size=current_loaded_data.shape[1]-1
        elif (x=='c'):
            current_working_architecture=define_architecture(current_data_size)
        elif (x=='d'):
            current_working_architecture=load_architecture()
        elif (x=='e'):
            current_working_weights=initialize_weights(current_working_architecture)
        elif (x=='f'):
            current_working_weights=load_weights()
        elif(x=='g'):
            save_architecture(current_working_architecture)
        elif(x=='h'):
            save_weights(current_working_weights)
        elif (x=='i'):
            learning_parameters=set_training_parameters()
        elif(x=='j'):
            save_training_parameters(learning_parameters)
        elif (x=='k'):
            learning_parameters=load_training_parameters()
        elif (x=='l'):
            choice = 1
        elif (x=='m'):
            current_working_weights, loss_history, accuracy_history= train_model(current_loaded_data, current_working_architecture, current_working_weights, learning_parameters, loss_history, accuracy_history)
        elif (x=='n'):
            loss_history, accuracy_history= test_data(current_loaded_data, current_working_architecture, current_working_weights, learning_parameters)
        elif (x=='o'):
            print_out(accuracy_history, "accuracy")
        elif (x=='p'):
            print_out(loss_history, "Loss")
    


In [2]:
def print_out(history, name):
    x=[]
    array_hy=np.shape(np.array(history))
    hy_size=np.squeeze(array_hy)
    print(hy_size)
    for y in range(0,hy_size):
        x.append(y*100)
    plt.plot(x, history)
    plt.xlabel("Epochs")
    plt.ylabel(name)
    plt.title(name)
    plt.show()
    

In [3]:
#This function loads data to be modeled from a file and vectorizes it, distinguishing between the input and the output
def load_data():
    print("\nPlease provide the file name of the csv that you would like to read as data.\n")
    print("Each line of this File should represent a single data point, and the first entry of each line should indicate to what class a given data point belongs\n")
    print("Your selection should include the .csv as its extension\n")
    filename=input("Filename: ")
    return load_particular_data(filename)

          

    

In [4]:
#This function allows the user to choose their loss function, learning rate, and l2 regularizer hyperparameters
def set_training_parameters():
    chosen_learning_rate=float(input("What learning rate would you like to use?"))
    chosen_l2_regularizer=float(input("What hyperparameter would you like to use for the\n L2 regularizer? (If no regularizer is to be used, hit '0'"))
    print("The options for loss functions are as follows:")
    print("a.) cross entropy with softmax")
    print("b.) hinge loss")
    loss_choice=input("Your choice: ")
    if (loss_choice=='a'):
        loss_function="cross_entropy"
    elif (loss_choice=='b'):
        loss_function="hinge_loss"
        
    training_parameters={"l2_regularizer": chosen_l2_regularizer,
        "learning_rate": chosen_learning_rate, 
        "loss_function":loss_function
    }
    return training_parameters

In [5]:
#This function saves the currently loaded learning parameters to a file
def save_training_parameters(parameters):
    file_name=input("What filename would you like to save the training parameters to?")
    parameter_list=[parameters]
    open_file=open(file_name,"wb")
    pickle.dump(parameter_list,open_file)
    open_file.close()

In [6]:
#This function loads the currently loaded learning parameters to a file
def load_training_parameters():
    filename=input("Please give the filename of the training parameters you would like to load")
    open_file=open(filename, "rb")
    loaded_list=pickle.load(open_file)
    open_file.close()
    for x in loaded_list:
        return x

In [7]:
#This function loads data from a predetermined filename
def load_particular_data(filename):
    raw_data=open(filename, 'rt')
    reader=csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
    x=list(reader)
    convertedx=np.array(x)
    data=convertedx.astype(np.float64)
    return data

In [8]:
#THIS FUNCTION PARTRITIONS THE CURRENTLY LOADED DATA AND SAVES IT TO MULTIPLE FILES THEN GIVES THE USER THE OPTION TO START WORKING WITH THE DIFFERENT DATA
def partition_current_loaded_data(data):
    print("\nThis module partitions the current loaded data into training, testing, and validation datasets")
    print("and then saves each of those datasets to csv files for future use. At the end of the use of this")
    print("module, you will be asked whether or not you want to change your currently loaded data to the new")
    print("one of the newly created data files, or if you want to stick with the loaded data you had at the ")
    print("time this function was called.\n\n")
    
    correct='0'
    while correct=='0':
        trainingpercent=float(input("Enter the proportion of the data you want to go into the training dataset as a number between 0 and 1,and then press enter"))
        testingpercent=float(input("Enter the proportion of the data you want to go into the testing dataset as a number between 0 and 1 and then press enter"))
        printabletraining=trainingpercent*100
        printabletesting=testingpercent*100
        printablevalidation=100-printabletraining-printabletesting
        print("You have chosen the following approximate proportions")
        print("Training:   "+ str(printabletraining)+ "%")
        print("Testing:    "+str(printabletesting)+ "%")
        print("Validation: "+ str(printablevalidation)+ "%")
        
        print("\n\n")
        correct=input("Is This Correct? \n (0) no \n (1) yes")
    newTraining=[]
    newTesting=[]
    newValidation=[]        
    for x in data:
        randomnumber=np.random.rand()            
        if (randomnumber>1-trainingpercent):
            newTraining.append(x)
        elif (randomnumber>1-trainingpercent-testingpercent):
           newTesting.append(x)
        elif (randomnumber>0):                
            newValidation.append(x)
                
                
    convertedTraining=np.array(newTraining)
    convertedTesting=np.array(newTesting)        
    convertedValidation=np.array(newValidation)
        
    print(convertedTraining.shape)        
    print(convertedTesting.shape)
    print(convertedValidation.shape)
        
    chosen_name=input("Type the name you would like to save these datasets as: ")
    trainingname=chosen_name+"TRAINING.csv"
    testingname=chosen_name+"TESTING.csv"
    validationname=chosen_name+"VALIDATION.csv"
    np.savetxt(trainingname, convertedTraining, delimiter=',')
    np.savetxt(testingname, convertedTesting, delimiter=',')
    np.savetxt(validationname, convertedValidation, delimiter=',')
    print("What data would you like to be your active data at the end of the execution of this function?")
    print("a: Previous Data")
    print("b: Partitioned Training Data")
    print("c: Partitioned Testing Data")
    print("d: Partitioned Validation Data")
    your_choice=input("Choice: ")
    if (your_choice=='a'):
        return data
    elif (your_choice=='b'):
        return convertedTraining
    elif (your_choice=='c'):
         return convertedTesting
    elif (your_choice=='d'):
        return convertedValidation

In [9]:
#This module lets the user define the architectrure of their neural net, including the activation function of each layer, numbe rof layers and number of neurons in each layer
#it is worht indicating that since softmax isn't considered as an actiobvation function here because it isn't actually an activation function. Instead, the softmax is considered a part of various loss functions
def define_architecture(inputdim):
    numlayers=int(input("How Many layers would you  like the neural net to have, including the output layer, starting counting at the first hidden layer?"))
    numclasses=int(input("How many classes would you like to classify your data into?"))
    layerinputdim=inputdim
    print("The currently loaded dataset is ")
    print(inputdim)
    print("dimensionsal.")
    print("For this reason the input dimension of layer 1 will be automatically set to equal")
    print(inputdim)
    created_architecture=[]
    for x in range(1,numlayers):
        print("For layer ")
        print(x)
        print("Please select your desired activation function.")
        print("a.) relu (Rectified linear unit)")
        print("b.) sigmoid")
        print("c.) linear")
        activation_choice=input("Your Choice: ")
        if(activation_choice == 'a'):
            act="relu"
        elif(activation_choice =='b'):
            act="sigmoid"
        elif(activation_choice=='c'):
            act="linear"
        layeroutputdim=int(input("Choose an output dimensionality: "))
        current_layer={"input_dim": layerinputdim, "output_dim": layeroutputdim, "activation": act}
        created_architecture.append(current_layer)
        layerinputdim=layeroutputdim
    print("For the final layer,")
    print("please select your desired activation function.")
    print("a.) relu (Rectified linear unit)")
    print("b.) sigmoid")
    print("c.) linear")
    activation_choice=input("Your Choice: ")
    if(activation_choice == 'a'):
        act="relu"
    elif(activation_choice =='b'):
        act="sigmoid"
    elif(activation_choice=='c'):
        act="linear"
    current_layer={"input_dim": layerinputdim, "output_dim": numclasses, "activation": act}
    created_architecture.append(current_layer)
    print(created_architecture)
    return created_architecture

In [10]:
#This function lets the user load an architecture from a previous session
def load_architecture():
    filename=input("Please give the filename of the architecture you would like to load")
    open_file=open(filename, "rb")
    loaded_list=pickle.load(open_file)
    open_file.close()
    return loaded_list

In [11]:
#This function lets the user load weights from a previous session
def load_weights():
    filename=input("Please give the filename of the architecture you would like to load")
    open_file=open(filename, "rb")
    loaded_list=pickle.load(open_file)
    open_file.close()
    return loaded_list

In [12]:
#This function lets the user save architectures for a different session
def save_architecture(architecture):
    file_name=input("What filename would you like to save the architecture to?")
    open_file=open(file_name,"wb")
    pickle.dump(architecture,open_file)
    open_file.close()

In [13]:
#This function lets the user save weights for a different session
def save_weights(weights):
    file_name=input("What filename would you like to save the weights to?")
    open_file=open(file_name,"wb")
    pickle.dump(weights,open_file)
    open_file.close()

In [14]:
#This is the cell in which I will put the various activation functions and how they behave upon backwards propogation. 

def sigmoid(x):
    return 1/(1+np.exp(-x))
    
def relu(x):
    return np.maximum(0,x)

def linear(x):
    return x


def linear_backward(dA,x):
    dx=np.array(dA,copy =True)
    return dx;

def relu_backward(dA,x):
    dx=np.array(dA,copy =True)
    dx[x<=0]=0;
    return dx;

def sigmoid_backward(dA,x):
    sig=sigmoid(x)
    return dA*sig*(1-sig)


    

In [15]:
#This is the cell in which I implement "Initialize Weights and Biases"
def initialize_weights(architecture,seed=105):
    np.random.seed(seed)
    number_of_layers=len(architecture)
    weight_values={}
    print("\nWould you like the initialized weights to have a Gaussian distribution or a uniform distribution?\n")
    print("a: Gaussian\n")
    print("b: Uniform\n")
    choice=input("Distribution Choice: ")
    print("\nWhat would you  like to be the variance of the weights you initialize?\n")
    variance=float(input("Chosen Variance: "))
    for identifier, layer in enumerate(architecture):
        layer_identifier=identifier+1
        layer_input_size=layer["input_dim"]
        layer_output_size=layer["output_dim"]
        if(choice=='a'):
            weight_values['W'+str(layer_identifier)]=variance*np.random.randn(layer_output_size, layer_input_size)
            weight_values['b'+str(layer_identifier)]=np.random.randn(layer_output_size, 1)*variance
        elif(choice=='b'):
            weight_values['W'+str(layer_identifier)]=np.random.rand(layer_output_size, layer_input_size)*variance
            weight_values['b'+str(layer_identifier)]=np.random.rand(layer_output_size, 1)*variance
    return weight_values
            
  
    

In [16]:
#In this cell, I implement a single step of forward propogation
def single_layer_forward_propogation(A_previous, W_current, b_current, activation):     
    x_current=np.transpose(np.dot(W_current,np.transpose(A_previous))+b_current)
    if activation == "relu":
        activation_function=relu
    elif activation == "sigmoid":
        activation_function = sigmoid
    elif activation == "linear":
        activation_function=linear
        
    return activation_function(x_current), x_current




In [17]:
#In this cell, I implement full forward propogation through the network
def full_forward_propogation(x, weight_values, architecture):
    memory={}
    A_current=x
    for identifier, layer in enumerate(architecture):
        layer_identifier=identifier+1
        A_previous=A_current
        current_activation_function=layer["activation"]
        W_current=weight_values["W"+str(layer_identifier)]
        b_current=weight_values["b"+str(layer_identifier)]
        A_current,  x_current=single_layer_forward_propogation(A_previous, W_current, b_current, layer["activation"])
        memory["A"+str(identifier)]=A_previous
        memory["X"+str(layer_identifier)]=x_current
    return A_current, memory

In [18]:
    
#This function calculates the loss function for cross entropy with a softmax

def get_loss(y_hat, y, learning_parameters):
    if learning_parameters["loss_function"]=="cross_entropy":
        return cross_entropy_with_softmax(y_hat,y)
    elif learning_parameters["loss_function"]=="hinge_loss":
        return hinge_loss(y_hat,y)
    return 0

def cross_entropy_with_softmax(y_hat, y):
    y_hat_softmax=softmax(y_hat)
    m=np.shape(y_hat_softmax)[1]
    return (-1/m)*(np.tensordot(y,np.log(y_hat_softmax)))
def cross_entropy_dA(y_hat, y):
    y_hat_softmax=softmax(y_hat)
    return y_hat_softmax-y
        
def hinge_loss(y_hat, y):
    dood=np.transpose(np.array([np.transpose(np.array(y_hat)),np.transpose(np.array(y))]))
    losses=[]
    for point in dood:
        other_point=np.transpose(point)
        y_hat_sub=other_point[0]
        y_sub=other_point[1]
        losses.append(max(0,1-np.dot(y_hat_sub,y_sub)+max(y_hat_sub-y_hat_sub*y_sub)))
    return sum(losses)/np.shape(np.array(y_hat))[0] 


def hinge_loss_dA(y_hat,y):
    dood=np.transpose(np.array([np.transpose(np.array(y_hat)),np.transpose(np.array(y))]))
    losses=[]
    gradients=[]
    for point in dood:
        other_point=np.transpose(point)
        y_hat_sub=other_point[0]
        y_sub=other_point[1]
        if(0==max(0,1-np.dot(y_hat_sub,y_sub)+max(y_hat_sub-y_hat_sub*y_sub))):
            gradients.append(np.zeros(np.shape(y_sub)))
        else:
            gradients.append(-y_sub+np.where(y_hat_sub==max(y_hat_sub),1,0))
    return np.array(gradients)

In [19]:
def softmax(y):
    soft_dood=[]
    for dood in y:
        dood_exp=np.exp(dood)
        sum_exp=np.sum(dood_exp)
        soft_dood.append(dood_exp/sum_exp)
    return np.array(soft_dood)

In [20]:
def single_layer_backward_propogation(dA_current, w_current, b_current, x_current, A_previous, layer):
    m=A_previous.shape[1]
    if layer["activation"] == "relu":
        dx_current=relu_backward(dA_current,x_current)
    elif layer["activation"] == "sigmoid":
        dx_current=sigmoid_backward(dA_current,x_current)
    elif layer["activation"] == "linear":
        dx_current=linear_backward(dA_current,x_current)
    dw_current=np.dot(np.transpose(A_previous),dx_current)/m
    db_current=np.sum(dx_current,axis=0,keepdims=True)/m
    dA_previous=np.dot(dx_current,w_current)
    
    return dA_previous, dw_current, db_current

In [21]:

def full_backward_propogation(y_hat, y, memory, weights, architecture, learning_parameters):
    grads_values = {}
    m = y.shape[1]
    y = y.reshape(y_hat.shape)
    if learning_parameters["loss_function"] == "cross_entropy":
        dA_previous =cross_entropy_dA(y_hat,y)
    elif learning_parameters["loss_function"] == "hinge_loss":
        dA_previous=hing_loss_dA(y_hat,y)

    gradient_current={}
    for layer_id_previous, layer in reversed(list(enumerate(architecture))):
        layer_id_current = layer_id_previous + 1
        current_activation_function = layer["activation"]
        
        dA_current = dA_previous
        
        A_previous = memory["A" + str(layer_id_previous)]
        x_current = memory["X" + str(layer_id_current)]
        w_current = weights["W" + str(layer_id_current)]
        b_current = weights["b" + str(layer_id_current)]
        
        dA_previous, dw_current, db_current = single_layer_backward_propogation(
            dA_current, w_current, b_current, x_current, A_previous, layer)
        
        gradient_current["dW" + str(layer_id_current)] = dw_current
        gradient_current["db" + str(layer_id_current)] = db_current
    
    return gradient_current

In [22]:
def update(weights, gradient, architecture, learning_parameters,batch_size):
    for layer_id, layer in enumerate(architecture):
        layerd=layer_id+1
        weights["W" + str(layerd)] -= (np.transpose(learning_parameters["learning_rate"] * gradient["dW" + str(layerd)])+learning_parameters["l2_regularizer"]*weights["W" + str(layerd)])  
        weights["b" + str(layerd)] -= (np.transpose(learning_parameters["learning_rate"] * gradient["db" + str(layerd)]))

    return weights;

In [23]:
def train_model(data_to_model, architecture, weights, learning_parameters, cost_history, accuracy_history):
    batch_size=int(input("How many data points would you like to make up a single batch.\n Note that only full batches will be trained."))
    epochnumber=int(input("How many epochs would you like to train for each batch?"))
    ### Splitting up data to model into batches
    data_batches=[]
    batch_being_built=[]
    some_number=1
    initial_y=[]
    for data_points in data_to_model:
        initial_y.append(np.array(data_points[0]))
    num_classes=int(np.amax(initial_y)+1)
    
    for x in data_to_model:
        batch_being_built.append(x)
        if (batch_size==some_number):
            batch_to_add=np.array(batch_being_built).astype(np.float64)
            data_batches.append(batch_to_add)
            batch_being_built=[]
            some_number=0
        some_number+=1
        
    loss_history=[]
    accuracy_history=[]
    n=0
    for current_batch in data_batches:
        print("starting a batch")
        input_doods, output_doods=split_data(current_batch, num_classes)
        for i in range(0, epochnumber):
            y_hat, cache = full_forward_propogation(input_doods, weights, architecture)
            loss_value=get_loss(y_hat, output_doods, learning_parameters)
            n+=1
            if(n==100):
                n=0
                print(y_hat)
                accuracy=get_accuracy_value(y_hat,output_doods,np.squeeze(np.shape(output_doods)[1]))
                print("Accuracy")
                print(accuracy)
                print("loss_value")
                print(loss_value)
                quitnow=input("quit, y or n")
                accuracy_history.append(accuracy)
                loss_history.append(loss_value)
                if (quitnow=='y'):
                    doodle=input("Really Quit (y), or just change training parameters (n)?")
                    if(doodle=='y'):
                        return weights,loss_history, accuracy_history
                    set_training_parameters()
            gradients=full_backward_propogation(y_hat,output_doods,cache,weights,architecture, learning_parameters)
            weights=update(weights,gradients,architecture,learning_parameters,batch_size)
        accuracy=get_accuracy_value(y_hat,output_doods, np.squeeze(np.shape(output_doods)[1]))
    return weights, loss_history, accuracy_history

In [24]:
def get_accuracy_value(y_hat,output_doods,num_classes):
    maxes=[]
    for x in y_hat:
        max_value=max(x)
        maxes.append(x.tolist().index(max_value))
    y_vector=np.array(maxes)
    right_size_zeros=np.zeros((int(np.squeeze(np.shape(y_vector))), num_classes))
    for stuff in range(0,np.squeeze(np.shape(y_vector))):
        right_size_zeros[stuff][int(np.floor(.5+y_vector[stuff]))]=1
    num_correct=np.tensordot(right_size_zeros,output_doods)
    portion_correct=num_correct/np.shape(output_doods)[0]
    print("number correct")
    print(num_correct)
    print("Accuracy")
    print(portion_correct)
    print("End_Accuracy")
    return portion_correct



In [25]:
def split_data(data,num_classes):
    initial_x=[]
    initial_y=[]
    m=np.squeeze(np.array(data[0]).shape)
    for data_points in data:
        initial_y.append(np.array(data_points[0]))
        initial_x.append(np.array(data_points[1:]))
    
    y_vector=np.array(initial_y)
    right_size_zeros=np.zeros((int(np.squeeze(np.shape(y_vector))), num_classes))
    for stuff in range(0,np.squeeze(np.shape(y_vector))):
        right_size_zeros[stuff][int(np.floor(.5+y_vector[stuff]))]=1
    x_vector=np.array(initial_x)

    return x_vector, right_size_zeros
        

In [26]:
def test_data(data_to_test, architecture, weights, learning_parameters):
    batch_size=int(input("How many data points would you like to test per batch"))
    data_batches=[]
    batch_being_built=[]
    some_number=1
    initial_y=[]
    for data_points in data_to_test:
        initial_y.append(np.array(data_points[0]))
    num_classes=int(np.amax(initial_y)+1)
    for x in data_to_test:
        batch_being_built.append(x)
        if (batch_size==some_number):
            batch_to_add=np.array(batch_being_built).astype(np.float64)
            data_batches.append(batch_to_add)
            batch_being_built=[]
            some_number=0
        some_number+=1
    accuracy_history=[]
    loss_history=[]
    n=0
    for current_batch in data_batches:
        input_doods, output_doods=split_data(current_batch, num_classes)
        y_hat,cache=full_forward_propogation(input_doods, weights, architecture)
        loss_value=get_loss(y_hat,output_doods,learning_parameters)
        accuracy=get_accuracy_value(y_hat, output_doods, np.squeeze(np.shape(output_doods)[1]))
        print("Accuracy")
        print(accuracy)
        print("loss_value")
        print(loss_value)
        accuracy_history.append(accuracy)
        loss_history.append(loss_value)
    print("Loss History")
    print(loss_history)
    print("Accuracy_history")
    print(accuracy_history)
    total=sum(accuracy_history)
    length=len(accuracy_history)
    print("Overall Accuracy")
    print(total/length)
    
    return loss_history, accuracy_history


In [ ]:




## #This is the "main" function of the multi-layer Perceptron-inator 3000, and just initializes the loop in which the user  chooses what tasks to performs
#

#Imports
import csv
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
print("Welcome To Multi-Layer Perceptron-inator 3000\n")
AskUserWhatToDo()

Welcome To Multi-Layer Perceptron-inator 3000



Select an option by typing the Indicated letter and pressing  "Enter" 


a:  Load csv of data to model
b: Partition  loaded data into Training, Testing, and Validation
c: Manually Define MLP Architecture which is compatible with loaded data
d: Load MLP Architecture From Previous Session which is compatible with loaded data
e: Initialize weights and biases for use with training
f: Load weights and biases from previous session
g: Save Current Architecture For Future Session
h: Save Current Weights For Future Session
i: Choose Training Parameters
j: Save Training Parameters
k: Load Training Parameters
l: Quit Program Without Saving
m: Train Model
n: Test Models
o: Print Accuracy History
p: Print Loss History


Your Choice:  a



Please provide the file name of the csv that you would like to read as data.

Each line of this File should represent a single data point, and the first entry of each line should indicate to what class a given data point belongs

Your selection should include the .csv as its extension



Filename:  mnist_train.csv




Select an option by typing the Indicated letter and pressing  "Enter" 


a:  Load csv of data to model
b: Partition  loaded data into Training, Testing, and Validation
c: Manually Define MLP Architecture which is compatible with loaded data
d: Load MLP Architecture From Previous Session which is compatible with loaded data
e: Initialize weights and biases for use with training
f: Load weights and biases from previous session
g: Save Current Architecture For Future Session
h: Save Current Weights For Future Session
i: Choose Training Parameters
j: Save Training Parameters
k: Load Training Parameters
l: Quit Program Without Saving
m: Train Model
n: Test Models
o: Print Accuracy History
p: Print Loss History


...

[[7.66312387e-01 3.30470183e-01 6.04321790e-04 ... 2.49090467e-01
  1.94939502e-04 2.60870900e-01]
 [8.01584840e-01 3.01284858e-01 1.91669764e-04 ... 2.18136846e-01
  5.18156687e-05 2.28875812e-01]
 [6.99706718e-01 3.80166430e-01 4.72713585e-03 ... 3.11928590e-01
  2.09665368e-03 3.21939987e-01]
 ...
 [7.58583669e-01 3.36108753e-01 7.98199589e-04 ... 2.57060117e-01
  2.68889754e-04 2.68442910e-01]
 [7.77418734e-01 3.17104167e-01 4.40553185e-04 ... 2.40661878e-01
  1.35646317e-04 2.52377205e-01]
 [7.24217532e-01 3.63156985e-01 2.10595386e-03 ... 2.86710653e-01
  8.23500481e-04 2.98671647e-01]]
number correct
18.0
Accuracy
0.140625
End_Accuracy
Accuracy
0.140625
loss_value
28.734686672030236


quit, y or n n


[[7.65584041e-01 3.30687521e-01 5.26251936e-04 ... 2.48981609e-01
  1.76429530e-04 2.60901336e-01]
 [8.01346356e-01 3.00935113e-01 1.62660689e-04 ... 2.17918620e-01
  4.59498961e-05 2.28623615e-01]
 [6.98963642e-01 3.80421357e-01 4.28571704e-03 ... 3.11903749e-01
  1.95441232e-03 3.22047824e-01]
 ...
 [7.57846761e-01 3.36333196e-01 6.99270721e-04 ... 2.56974031e-01
  2.44491652e-04 2.68489827e-01]
 [7.76776876e-01 3.16866398e-01 3.81987531e-04 ... 2.40635467e-01
  1.22475832e-04 2.52455998e-01]
 [7.22996745e-01 3.63826534e-01 1.88605375e-03 ... 2.86775326e-01
  7.61961591e-04 2.99023032e-01]]
number correct
18.0
Accuracy
0.140625
End_Accuracy
Accuracy
0.140625
loss_value
28.729937525760533


quit, y or n n


[[7.64758259e-01 3.30951671e-01 4.61998762e-04 ... 2.48868660e-01
  1.60549814e-04 2.60950661e-01]
 [8.01069755e-01 3.00558139e-01 1.39304256e-04 ... 2.17694712e-01
  4.09951983e-05 2.28363258e-01]
 [6.98140847e-01 3.80701313e-01 3.90819035e-03 ... 3.11876096e-01
  1.82898347e-03 3.22171699e-01]
 ...
 [7.57006170e-01 3.36613178e-01 6.17447202e-04 ... 2.56884855e-01
  2.23493135e-04 2.68559014e-01]
 [7.76047730e-01 3.16630293e-01 3.34050832e-04 ... 2.40610694e-01
  1.11223544e-04 2.52556523e-01]
 [7.21630996e-01 3.64586392e-01 1.70090346e-03 ... 2.86839671e-01
  7.08366107e-04 2.99417828e-01]]
number correct
18.0
Accuracy
0.140625
End_Accuracy
Accuracy
0.140625
loss_value
28.724787901436272
